In [1]:
import sqlite3
import pandas as pd
import numpy as np

Récupération des données

In [2]:
conn = sqlite3.connect('../../db.sqlite3')

query_pizzas = """
SELECT pizza_id, name, size, ingredients
FROM main_pizza;
"""
pizzas_df = pd.read_sql_query(query_pizzas, conn)

query_orders = """
SELECT client_id, pizza_id, order_date
FROM main_order;
"""
orders_df = pd.read_sql_query(query_orders, conn)
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

conn.close()


Fonction de recommandation

In [3]:
def get_pizza_ingredients(pizza_id, pizzas_df):
    return pizzas_df[pizzas_df['pizza_id'] == pizza_id]['ingredients'].iloc[0].split(', ')

def recommend_pizzas_based_on_ingredients(client_pizzas, pizzas_df, top_k=1):
    """
    Recommande des pizzas basées sur les ingrédients des pizzas commandées par le client.
    
    Parameters:
        client_pizzas (List[str]): Liste des pizza_ids des pizzas commandées par le client.
        similarity_matrix (array): Matrice de similarité entre les pizzas.
        pizzas_df (DataFrame): Informations sur les pizzas.
        top_k (int): Nombre de recommandations.
    
    Returns:
        List[Tuple[str, float]]: Liste des recommandations avec scores.
    """
    # Extraire les ingrédients des pizzas commandées par le client
    client_ingredients = []
    for pizza_id in client_pizzas:
        client_ingredients.extend(get_pizza_ingredients(pizza_id, pizzas_df))
    client_ingredients = set(client_ingredients)  # Éviter les doublons
    
    # Calculer la similarité avec les autres pizzas basées sur les ingrédients partagés
    similarities = []
    for idx, pizza in pizzas_df.iterrows():
        pizza_ingredients = set(pizza['ingredients'].split(', '))
        common_ingredients = len(client_ingredients.intersection(pizza_ingredients))
        total_ingredients = len(client_ingredients.union(pizza_ingredients))
        similarity_score = common_ingredients / total_ingredients if total_ingredients > 0 else 0
        similarities.append((pizza['pizza_id'], similarity_score))
    
    # Trier les pizzas par similarité décroissante et retourner les k premières
    recommendations = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    recommended_pizzas = [pizza[0] for pizza in recommendations]
    
    return recommended_pizzas


Évaluation des performances

In [4]:
def precision_recall_at_k(recommended_pizzas, relevant_pizzas, k=1):
    recommended_set = set(recommended_pizzas[:k])  
    relevant_set = set(relevant_pizzas)  # Pizzas réellement commandées
    hits = recommended_set.intersection(relevant_set)
    
    precision = len(hits) / k
    recall = len(hits) / len(relevant_set) if len(relevant_set) > 0 else 0
    return precision, recall

clients = orders_df['client_id'].unique()

precisions, recalls, f1_scores = [], [], []

for client_id in clients:
    relevant_pizzas = orders_df[orders_df['client_id'] == client_id]['pizza_id'].tolist()
    relevant_pizzas = pizzas_df[pizzas_df['pizza_id'].isin(relevant_pizzas)]['name'].tolist()
    
    client_pizzas = orders_df[orders_df['client_id'] == client_id]['pizza_id'].tolist()
    
    # Recommandations basées sur ingrédients
    recommended_pizzas = recommend_pizzas_based_on_ingredients(client_pizzas, pizzas_df, top_k=1)
    recommended_pizzas_names = pizzas_df[pizzas_df['pizza_id'].isin(recommended_pizzas)]['name'].tolist()
    
    print(f"Client {client_id} :")
    print(f"Recommandations : {recommended_pizzas_names}")
    print(f"Pizzas pertinentes : {relevant_pizzas}")
    
    precision, recall = precision_recall_at_k(recommended_pizzas_names, relevant_pizzas, k=1)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)

print(f"Précision moyenne : {sum(precisions)/len(precisions):.2f}")
print(f"Rappel moyen : {sum(recalls)/len(recalls):.2f}")
print(f"F1-Score moyen : {sum(f1_scores)/len(f1_scores):.2f}")


Client 1 :
Recommandations : ['The Mexicana Pizza']
Pizzas pertinentes : ['The Spinach Pesto Pizza', 'The Spinach Supreme Pizza', 'The Greek Pizza', 'The Hawaiian Pizza', 'The Spinach and Feta Pizza', 'The Classic Deluxe Pizza', 'The Mexicana Pizza']
Client 2 :
Recommandations : ['The Greek Pizza']
Pizzas pertinentes : ['The Greek Pizza', 'The Greek Pizza', 'The Brie Carre Pizza', 'The Greek Pizza', 'The Greek Pizza', 'The Five Cheese Pizza']
Client 3 :
Recommandations : ['The Spinach Supreme Pizza']
Pizzas pertinentes : ['The Thai Chicken Pizza', 'The Pepperoni, Mushroom, and Peppers Pizza', 'The Greek Pizza', 'The Hawaiian Pizza', 'The Thai Chicken Pizza', 'The Mediterranean Pizza', 'The Pepper Salami Pizza']
Client 4 :
Recommandations : ['The Mexicana Pizza']
Pizzas pertinentes : ['The Spicy Italian Pizza', 'The Calabrese Pizza', 'The Pepperoni Pizza', 'The Vegetables + Vegetables Pizza', 'The Spinach and Feta Pizza', 'The Pepperoni, Mushroom, and Peppers Pizza', 'The California Chi